In [1]:
import pandas as pd
import numpy as np

# Read elc stock data into a dataframe
elc_stock_df = pd.read_excel('data_files/elc_stock.xlsx')

# manufacturing information
manuf = pd.read_excel('data_files/manuf.xlsx')
manuf['Model Number'] = manuf['Model Number'].astype(str)

In [2]:
elc_stock_df_reserve = elc_stock_df.copy()

In [291]:
elc_stock_df = elc_stock_df_reserve.copy()

In [3]:
elc_stock_df['MS Code'] = elc_stock_df['MS Code'].fillna(elc_stock_df['Material'])

In [4]:
# Drop rows where column MS Code is NaN
elc_stock_df = elc_stock_df.dropna(subset=['MS Code'])
elc_stock_df['MS Code'] = elc_stock_df['MS Code'].astype(str)
elc_stock_df['Material'] = elc_stock_df['Material'].astype(str)
elc_stock_df['BU'] = elc_stock_df['Product hierarchy'].str.slice(stop=5)

In [5]:
elc_stock_df = elc_stock_df.merge(manuf, left_on='Material', right_on='Model Number', how='left')

In [6]:
elc_stock_df = elc_stock_df[elc_stock_df['Material'].str.contains('_00') | (elc_stock_df['Current Free Stock for Ordering OpCo'] != 0) | (elc_stock_df['Safety Stock'] != 0)]


elc_stock_df_l = elc_stock_df.loc[:, ['MS Code', 'Manuf.', 'BU', 'Current Free Stock for Ordering OpCo', 'Safety Stock']]
#elc_stock_df_l = elc_stock_df_l.loc[(elc_stock_df_l['Current Free Stock for Ordering OpCo'] != 0) | (elc_stock_df_l['Safety Stock'] != 0)]

In [7]:
# Drop rows where column MS Code is NaN
elc_stock_df_l = elc_stock_df_l.dropna(subset=['MS Code'])

In [8]:
key_bu = ['YY112', 'YY113', 'YY114', 'YY115', 'YY116', 'YY117','YY118', 'YY119', 'YY138']

elc_stock_df_l = elc_stock_df_l[elc_stock_df_l['BU'].isin(key_bu)]

In [9]:
elc_stock_df_l = elc_stock_df_l[~elc_stock_df_l['MS Code'].str.contains('BOP')]

In [10]:
elc_stock_df_l = elc_stock_df_l.sort_values(by='Safety Stock', ascending=False)
elc_stock_df_l = elc_stock_df_l.drop_duplicates(subset='MS Code', keep='first')

In [11]:
# Write the DataFrame to an Excel file
elc_stock_df_l.to_excel('data_files/output3.xlsx', index=False)

In [12]:
# Read orders data
orders = pd.read_excel('data_files/orders_data.xlsx')

In [36]:
orders_df = orders.copy()

In [37]:
len(orders_df)

530636

In [38]:
# Delete rows where 'ms_code' contains text with 'BOP'
orders_df['ms_code'] = orders_df['ms_code'].astype(str)
orders_df = orders_df[~orders_df['ms_code'].str.contains('BOP')]

# Delete rows where 'ms_code' contains text with 'ENT'
orders_df['ms_code'] = orders_df['ms_code'].astype(str)
orders_df = orders_df[~orders_df['ms_code'].str.contains('ENT')]

# Delete rows where both column 'a' and column 'b' values are 0
#orders_df = orders_df.loc[(orders_df['order_intake_quantity'] != 0) | (orders_df['order_intake_EUR'] != 0)]

#key_opcos = ['YEF-G', 'YEF-A', 'YEF-CH']
key_bu = ['YY112', 'YY113', 'YY114', 'YY115', 'YY116', 'YY117','YY118', 'YY119', 'YY138']

# Keep rows where column 'd' values match the list of values
#orders_df = orders_df[orders_df['company_code_n'].isin(key_opcos)]

# Keep rows where column 'd' values match the list of values
orders_df = orders_df[orders_df['bu'].isin(key_bu)]
orders_df.reset_index(inplace=True, drop=True)

In [39]:
# add grouping tag for 5 month

monthes = [201804, 201805, 201806, 201807,
       201808, 201809, 201810, 201811, 201812, 201901, 201902, 201903,
       201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911,
       201912, 202001, 202002, 202003, 202004, 202005, 202006, 202007,
       202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103,
       202104, 202105, 202106, 202107, 202108, 202212, 202204, 202112,
       202109, 202303, 202210, 202203, 202211, 202207, 202208, 202205,
       202209, 202111, 202301, 202206, 202201, 202202, 202110, 202302]

monthes = sorted(monthes)

orders_df_for_five = orders_df[orders_df['year_month'].isin(monthes)]
# Sort the column in ascending order
orders_df_for_five = orders_df_for_five.sort_values('year_month')
# Map each month to its group
month_to_group = {month: (i // 5) + 1 for i, month in enumerate(monthes)}
# Assign groups to 'monthes' in the DataFrame
orders_df_for_five['5_month'] = orders_df_for_five['year_month'].map(month_to_group)

In [40]:
# XYZ analysis

def mean_std_from_first_nonzero(row):
    # Find the index of the first non-zero value in the row
    nonzero_indices = np.nonzero(row.values[1:])[0]
    if len(nonzero_indices) > 0:
        first_nonzero_index = nonzero_indices[0] + 1  # Add 1 to account for the fact that we started at the second column
        mean = row.iloc[first_nonzero_index:].mean()  # Calculate the mean of the remaining values
        std = row.iloc[first_nonzero_index:].std()  # Calculate the standard deviation of the remaining values
        return pd.Series({'Mean': mean, 'Std': std})  # Return the mean and standard deviation as a Series
    else:
        return pd.Series({'Mean': 0, 'Std': 0})  # If all values are zero, return 0 for mean and standard deviation

In [19]:
# create a pivot table for ms_code for each month with NaN values filled with 0
pivot_table_month = pd.pivot_table(orders_df, values='order_intake_quantity', index='ms_code', columns='year_month_date', aggfunc='sum', fill_value=0) # margins=True , fill_value=0
pivot_table_month.reset_index(inplace=True)
# sort the pivot table by the total column in descending order
#pivot_table_month = pivot_table_month.sort_values(by='All', ascending=False)

pivot_table_half_year = pd.pivot_table(orders_df, values='order_intake_quantity', index='ms_code', columns=['FY', 'half_year'], aggfunc='sum', fill_value=0)
pivot_table_half_year.reset_index(inplace=True)

pivot_table_half_quarter = pd.pivot_table(orders_df, values='order_intake_quantity', index='ms_code', columns=['FY', 'quarter'], aggfunc='sum', fill_value=0)
pivot_table_half_quarter.reset_index(inplace=True)

pivot_table_five_month = pd.pivot_table(orders_df_for_five, values='order_intake_quantity', index='ms_code', columns=['5_month'], aggfunc='sum', fill_value=0)
pivot_table_five_month.reset_index(inplace=True)


# Calculate the sum of the last year for different pivots
last_12_sum = pivot_table_month.iloc[:, -12:].sum(axis=1)
last_2_sum = pivot_table_half_year.iloc[:, -2:].sum(axis=1)
last_4_sum = pivot_table_half_quarter.iloc[:, -4:].sum(axis=1)
last_f5_sum = pivot_table_five_month.iloc[:, -2:].sum(axis=1)


# Select the rows where the sum is greater than 0
pivot_table_month = pivot_table_month[last_12_sum > 0]
pivot_table_month.reset_index(inplace=True, drop=True)

pivot_table_half_year = pivot_table_half_year[last_2_sum > 0]
pivot_table_half_year.reset_index(inplace=True, drop=True)

pivot_table_half_quarter = pivot_table_half_quarter[last_4_sum > 0]
pivot_table_half_quarter.reset_index(inplace=True, drop=True)

pivot_table_five_month = pivot_table_five_month[last_f5_sum > 0]
pivot_table_five_month.reset_index(inplace=True, drop=True)


# calculate mean and std deviation for each product from the first month where it was sold
# Apply the function to each row of the dataframe, and store the results in new columns
pivot_table_month[['Mean_from_first', 'Std_from_first']] = pivot_table_month.apply(mean_std_from_first_nonzero, axis=1)
pivot_table_half_year[['Mean_from_first_hy', 'Std_from_first_hy']] = pivot_table_half_year.apply(mean_std_from_first_nonzero, axis=1)
pivot_table_half_quarter[['Mean_from_first_q', 'Std_from_first_q']] = pivot_table_half_quarter.apply(mean_std_from_first_nonzero, axis=1)
pivot_table_five_month[['Mean_from_first_5m', 'Std_from_first_5m']] = pivot_table_five_month.apply(mean_std_from_first_nonzero, axis=1)


# calculate mean and std deviation for each product
df_mean = pivot_table_month.iloc[:, 1:].mean(axis=1)
df_std = pivot_table_month.iloc[:, 1:].std(axis=1)

df_mean_h = pivot_table_half_year.iloc[:, 1:].mean(axis=1)
df_std_h = pivot_table_half_year.iloc[:, 1:].std(axis=1)

df_mean_q = pivot_table_half_quarter.iloc[:, 1:].mean(axis=1)
df_std_q = pivot_table_half_quarter.iloc[:, 1:].std(axis=1)

df_mean_5m = pivot_table_five_month.iloc[:, 1:].mean(axis=1)
df_std_5m = pivot_table_five_month.iloc[:, 1:].std(axis=1)


# categorize products based on std deviation
df_category = pd.cut(df_std, bins=[-float('inf'), 0.5*pivot_table_month['Mean_from_first'].mean(), pivot_table_month['Mean_from_first'].mean(), float('inf')], labels=['Z', 'Y', 'X'])
df_category_h = pd.cut(df_std_h, bins=[-float('inf'), 0.5*pivot_table_half_year['Mean_from_first_hy'].mean(), pivot_table_half_year['Mean_from_first_hy'].mean(), float('inf')], labels=['ZH', 'YH', 'XH'])
df_category_q = pd.cut(df_std_q, bins=[-float('inf'), 0.5*pivot_table_half_quarter['Mean_from_first_q'].mean(), pivot_table_half_quarter['Mean_from_first_q'].mean(), float('inf')], labels=['ZQ', 'YQ', 'XQ'])
df_category_5m = pd.cut(df_std_5m, bins=[-float('inf'), 0.5*pivot_table_five_month['Mean_from_first_5m'].mean(), pivot_table_five_month['Mean_from_first_5m'].mean(), float('inf')], labels=['Z5M', 'Y5M', 'X5M'])


# add new columns to the original dataframe
pivot_table_month = pivot_table_month.assign(Mean=df_mean, Std=df_std, Category=df_category)
pivot_table_month = pivot_table_month.loc[:,['ms_code','Mean_from_first', 'Std_from_first', 'Mean', 'Std', 'Category']]

pivot_table_half_year = pivot_table_half_year.assign(Mean_hy=df_mean_h, Std_hy=df_std_h, Category_hy=df_category_h)
pivot_table_half_year = pivot_table_half_year.loc[:,['ms_code','Mean_from_first_hy', 'Std_from_first_hy', 'Mean_hy', 'Std_hy', 'Category_hy']]
pivot_table_half_year = pivot_table_half_year.reset_index(level=0, drop=True)
pivot_table_half_year.columns = pivot_table_half_year.columns.get_level_values(0)


pivot_table_five_month = pivot_table_five_month.assign(Mean_5m=df_mean_5m, Std_5m=df_std_5m, Category_5m=df_category_5m)
pivot_table_five_month = pivot_table_five_month.loc[:,['ms_code','Mean_from_first_5m', 'Std_from_first_5m', 'Mean_5m', 'Std_5m', 'Category_5m']]
pivot_table_five_month = pivot_table_five_month.reset_index(level=0, drop=True)
pivot_table_five_month.columns = pivot_table_five_month.columns.get_level_values(0)


pivot_table_half_quarter = pivot_table_half_quarter.assign(Mean_q=df_mean_q, Std_q=df_std_q, Category_q=df_category_q)
pivot_table_half_quarter = pivot_table_half_quarter.loc[:,['ms_code','Mean_from_first_q', 'Std_from_first_q', 'Mean_q', 'Std_q', 'Category_q']]
pivot_table_half_quarter = pivot_table_half_quarter.reset_index(level=0, drop=True)
pivot_table_half_quarter.columns = pivot_table_half_quarter.columns.get_level_values(0)

writer = pd.ExcelWriter('data_files/order_data_XYZ.xlsx')
pivot_table_month.to_excel(writer, sheet_name='month')
pivot_table_half_year.to_excel(writer, sheet_name='half year')
pivot_table_half_quarter.to_excel(writer, sheet_name='quarter')
pivot_table_five_month.to_excel(writer, sheet_name='5months')
writer.close()

In [35]:
# Read price data
prices = pd.read_excel('data_files/priced_positions.xlsx')

In [41]:
prices= prices.loc[:, ['Model', 'COGS', 'Chk']]
orders_df = orders_df.merge(prices, left_on='ms_code', right_on='Model')

In [43]:
orders_data_opt = orders_df.copy()

In [44]:
# Creating an empty dataframe
final_df = pd.DataFrame()
column_A = 'ABC_Group'
value_A = 'A'
column_x = 'Category_XYZ_5M' #Category_XYZ_HY
value_X = 'X5M' #XH


# ABC + XYZ analysis per bu

# Create a new Excel writer object
writer = pd.ExcelWriter('data_files/order_data_ABC_XYZ_per_bu.xlsx')

# Get a list of the unique 'bu' values
bu_values = sorted(orders_data_opt['bu'].unique())

# Loop over each 'bu' value
for bu in bu_values:

    # Filter the data for the current 'bu' value
    bu_df = orders_data_opt[orders_data_opt['bu'] == bu]
    
    # prices 
    
    indexed_df = bu_df.set_index('ms_code')
    ms_code_price = indexed_df['COGS'].to_dict()
    ms_code_priced = pd.Series(ms_code_price)
    
    indexed_df_status = bu_df.set_index('ms_code')
    ms_code_status = indexed_df['Chk'].to_dict()
    ms_code_statused = pd.Series(ms_code_status)

    # Group by 'ms_code' and calculate the total 'order_intake_EUR'
    total_eur = bu_df.groupby('ms_code')['order_intake_EUR'].sum()
    total_quant = bu_df.groupby('ms_code')['order_intake_quantity'].sum()

    # Calculate the share of 'order_intake_EUR' for each 'ms_code'
    eur_share = total_eur / total_eur.sum()

    # Sort the 'ms_code' by descending 'order_intake_EUR'
    sorted_ms = total_eur.sort_values(ascending=False)

    # Calculate the cumulative sum of the sorted 'order_intake_EUR'
    cumulative_sum = sorted_ms.cumsum()

    # Calculate the percentage of the cumulative sum
    cumulative_percent = cumulative_sum / total_eur.sum() * 100

    # Categorize the 'ms_code' into ABC groups based on the cumulative percentage
    abc_group = pd.cut(cumulative_percent, bins=[0, 70, 90, 100], labels=['A', 'B', 'C'])

    # Create a new dataframe with the results
    results_df = pd.DataFrame({
        'Total_EUR': total_eur,
        'Share': eur_share,
        'Cumulative_Sum_EUR': cumulative_sum,
        'Cumulative_Percent': cumulative_percent,
        'ABC_Group': abc_group,
        'Quantity': total_quant,
        'COGS': ms_code_priced,
        'Status': ms_code_statused
    })
    

    # Write the results to a new sheet in the Excel file
    results_df.reset_index(inplace=True)
    results_df = results_df.rename(columns={'index': 'ms_code'})
    results_df = results_df.merge(pivot_table_month[['ms_code', 'Mean_from_first', 'Std_from_first', 'Category']],
                       on='ms_code', how='left')
    results_df= results_df.merge(pivot_table_half_quarter[['ms_code', 'Mean_from_first_q', 'Std_from_first_q', 'Category_q']],
                       on='ms_code', how='left')
    results_df = results_df.merge(pivot_table_half_year[['ms_code', 'Mean_from_first_hy', 'Std_from_first_hy', 'Category_hy']],
                       on='ms_code', how='left')
    results_df = results_df.merge(pivot_table_five_month[['ms_code', 'Mean_from_first_5m', 'Std_from_first_5m', 'Category_5m']],
                       on='ms_code', how='left')
    
    
    abc_xyz = results_df.loc[:,['ms_code', 'Total_EUR', 'ABC_Group',  'Category', 'Quantity', 'Mean_from_first',
       'Category_q', 'Mean_from_first_q', 'Category_hy', 'Mean_from_first_hy', 'Mean_from_first_5m', 'Std_from_first_5m', 'Category_5m', 'COGS', 'Status']]
    
    abc_xyz = abc_xyz.rename(columns={
        'ms_code': 'Model',
        'Mean_from_first': 'Mean',
        'Category': 'Category_XYZ',
        'Mean_from_first_q': 'Mean_Q',
        'Category_q': 'Category_XYZ_Q',
        'Mean_from_first_hy': 'Mean_HY',
        'Category_hy': 'Category_XYZ_HY',
        'Mean_from_first_5m': 'Mean_5M', 
         'Category_5m': 'Category_XYZ_5M',
        'Std_from_first_5m': 'Std_dev_5M'
    })
    
    abc_xyz['Quantity'] = abc_xyz['Quantity'].round(0)
    abc_xyz['Mean'] = abc_xyz['Mean'].round(0)
    abc_xyz['Mean_Q'] = abc_xyz['Mean_Q'].round(0)
    abc_xyz['Mean_HY'] = abc_xyz['Mean_HY'].round(0)
    abc_xyz['Mean_5M'] = abc_xyz['Mean_5M'].round(0)
    abc_xyz['Std_dev_5M'] = abc_xyz['Std_dev_5M'].round(3)
    
    abc_xyz = abc_xyz.sort_values(by=['ABC_Group', 'Category_XYZ', 'Total_EUR', 'Quantity'], ascending=[True, False, False, False])
    
    abc_xyz['sum'] = 0  # initialize 'sum' column to 0
    
    mask = (abc_xyz['ABC_Group'] == 'A') & (abc_xyz['Category_XYZ'] == 'X')  # create a boolean mask for the rows to multiply
    mask_q = (abc_xyz['ABC_Group'] == 'A') & (abc_xyz['Category_XYZ_Q'] == 'XQ')
    mask_hy = (abc_xyz['ABC_Group'] == 'A') & (abc_xyz['Category_XYZ_HY'] == 'XH')
    mask_5m = (abc_xyz['ABC_Group'] == 'A') & (abc_xyz['Category_XYZ_5M'] == 'X5M')
    
    mask2 = (abc_xyz['Status'] == 1)
    abc_xyz.loc[mask2, 'COGS'] = ((abc_xyz.loc[mask2, 'Total_EUR'])* 0.7 / abc_xyz.loc[mask2, 'Quantity'])
    

    abc_xyz.loc[mask, 'sum'] = abc_xyz.loc[mask, 'COGS'] * abc_xyz.loc[mask, 'Mean']  # multiply the selected rows and assign to 'sum' column
    abc_xyz.loc[mask_q, 'sum_q'] = abc_xyz.loc[mask_q, 'COGS'] * abc_xyz.loc[mask_q, 'Mean_Q']
    abc_xyz.loc[mask_hy, 'sum_hy'] = abc_xyz.loc[mask_hy, 'COGS'] * abc_xyz.loc[mask_hy, 'Mean_HY']
    abc_xyz.loc[mask_5m, 'sum_5m'] = abc_xyz.loc[mask_5m, 'COGS'] * abc_xyz.loc[mask_5m, 'Mean_5M']
    
    a_x_df = abc_xyz[(abc_xyz[column_A] == value_A)  & (abc_xyz['Category_XYZ'] == 'X')] #& (abc_xyz[column_x] == value_X)
    
    final_df = pd.concat([final_df, a_x_df], ignore_index=True)
    
    
    abc_xyz.to_excel(writer, sheet_name=bu)

# Save and close the Excel writer object
writer.close()

In [45]:
writer = pd.ExcelWriter('data_files/axfile.xlsx')
final_df.to_excel(writer, sheet_name='full')
writer.close()

In [46]:
elc_stock_df_l_r = elc_stock_df_l.copy()

In [324]:
elc_stock_df_l = elc_stock_df_l_r.copy()

In [47]:
elc_addition = final_df.copy()

In [48]:
elc_addition = elc_addition.rename(columns={'Model':'MS Code'})

In [49]:
elc_addition = elc_addition.loc[:, ['MS Code', 'Mean', 'Std_dev_5M', 'COGS']]

In [50]:
elc_stock_df_l = elc_stock_df_l.merge(elc_addition, on='MS Code', how='outer', indicator=True)

In [51]:
elc_stock_df_l['New safety stock level'] = np.round(elc_stock_df_l['Std_dev_5M'] * 1.65* np.sqrt(5), 0)

In [52]:
elc_stock_df_l['COGS'].fillna(0, inplace=True)

In [53]:
elc_stock_df_l['COGS'] = elc_stock_df_l['COGS'].astype(float)

In [54]:
elc_stock_df_l['New safety stock level'].fillna(0, inplace=True)

In [55]:
elc_stock_df_l['Mean'].fillna(0, inplace=True)

In [56]:
elc_stock_df_l['Current Free Stock for Ordering OpCo'].fillna(0, inplace=True)

In [57]:
elc_stock_df_l['Initial Order'] = elc_stock_df_l['New safety stock level'] +  5 * elc_stock_df_l['Mean'] - elc_stock_df_l['Current Free Stock for Ordering OpCo']

In [58]:
elc_stock_df_l['Budget Estimation'] = 1.104 * elc_stock_df_l['COGS'] * elc_stock_df_l['Initial Order']

In [59]:
elc_stock_df_l['Budget Estimation'] = elc_stock_df_l['Budget Estimation'].apply(lambda x: round(x, -1))

In [60]:
pivot_table_five_month = pd.pivot_table(orders_df_for_five, values='order_intake_quantity', index='ms_code', columns=['5_month'], aggfunc='sum', fill_value=0)
pivot_table_five_month.reset_index(inplace=True)

In [62]:
data = month_to_group.copy()

# Group the keys by values
groups = {}
for key, value in data.items():
    if value not in groups:
        groups[value] = []
    groups[value].append(key)

# Connect the first and last keys for each value as a string
result = {}
for value, keys in groups.items():
    first_key = keys[0]
    last_key = keys[-1]
    result[value] = str(first_key) + "-" + str(last_key)

pivot_table_five_month = pivot_table_five_month.rename(columns=result)

In [63]:
elc_stock_df_l = elc_stock_df_l.merge(pivot_table_five_month, left_on='MS Code', right_on='ms_code', how='left')

In [64]:
writer = pd.ExcelWriter('data_files/plus.xlsx')
elc_stock_df_l.to_excel(writer, sheet_name='elc+')
writer.close()